In [1]:
import pandas as pd
import dask.dataframe as dd

Load Trade Data
------------------

In [2]:
iter_csv = pd.read_csv('year_origin_destination_sitc_rev2.tsv', sep='\t', iterator=True, chunksize=100000)
trade = pd.concat([chunk[chunk['year'] == 2013] for chunk in iter_csv])
trade = trade[['year','origin','destination','export_val','import_val']].groupby(["year","origin","destination"]).sum()
#-Correct Country Codes-#
trade = trade.reset_index()
trade.origin = trade.origin.apply(lambda x: x.upper())
trade.destination = trade.destination.apply(lambda x: x.upper())

In [3]:
#-Exporter and Importer Reports-#
data = trade.copy()
ex = data[["year","origin","destination","export_val"]].rename(columns={'export_val':'value','origin':'eiso3c','destination':'iiso3c'})
im = data[["year","origin","destination","import_val"]].rename(columns={'import_val':'value','origin':'iiso3c','destination':'eiso3c'})

This demonstrates the availability of data. 
Trade between Ireland (IRL) and Iran (IRN) is captured predominantly from Ireland's reporting of exports and imports

In [17]:
trade.ix[(trade.origin == "IRN") & (trade.destination == "IRL")]

,year,origin,destination,export_val,import_val


In [18]:
trade.ix[(trade.origin == "IRL") & (trade.destination == "IRN")]

,year,origin,destination,export_val,import_val
11372,2013,IRL,IRN,75175445,1302978


Using Importer reports only sometimes excludes some trade flows. 

Exporter Reports

In [12]:
ex[(ex.eiso3c == "IRN") & (ex.iiso3c == "IRL")]

,year,eiso3c,iiso3c,value


In [13]:
ex[(ex.eiso3c == "IRL") & (ex.iiso3c == "IRN")]

,year,eiso3c,iiso3c,value
11372,2013,IRL,IRN,75175445


Importer Reports

In [10]:
im.ix[(im.eiso3c == "IRN") & (im.iiso3c == "IRL")]

,year,iiso3c,eiso3c,value
11372,2013,IRL,IRN,1302978


In [11]:
im.ix[(im.eiso3c == "IRL") & (im.iiso3c == "IRN")]

,year,iiso3c,eiso3c,value


Compute the rest of the data

In [15]:
#-Merge Importer Reported Data Together-#
dataset = im.copy()
#-Bilateral Relationships-#
dist = pd.read_excel('dist_cepii.xls')
bilat_attrs = ["iso_o","iso_d","contig","comlang_off","colony","dist","distcap","distw","distwces"]
print dist.shape
dist = dist[bilat_attrs].drop_duplicates()
print dist.shape
dataset = dataset.merge(dist[bilat_attrs], left_on=["eiso3c","iiso3c"], right_on=["iso_o","iso_d"], how="inner")
for item in ["iso_o","iso_d"]:
	del dataset[item]

(50176, 14)
(50176, 9)


Some items dropped from geo file due to the presence of duplicate rows

In [19]:
#-Country Attributes-#
geo = pd.read_excel('geo_cepii.xls')
cntry_attrs = ["iso3","landlocked"]
print geo.shape
geo = geo[cntry_attrs].drop_duplicates() 		#This is required as there are multiple entries for some countries such as AUS
print geo.shape
dataset = dataset.merge(geo[cntry_attrs], left_on="eiso3c", right_on="iso3", how="inner")
del dataset["iso3"]
dataset.rename(columns={'landlocked':'ell'}, inplace=True)
dataset = dataset.merge(geo[cntry_attrs], left_on="iiso3c", right_on="iso3", how="inner")
del dataset["iso3"]
dataset.rename(columns={'landlocked':'ill',}, inplace=True)

(238, 34)
(225, 2)


In [20]:
#-WDI-#
wdi = pd.read_csv('WDI_Data.csv')
idx = ["Country Code","Indicator Code"]
years = ["2013"]
wdi = wdi[idx+years]
wdi.columns = ["iso3c","code","value"]
wdi = wdi.set_index(["iso3c","code"]).unstack("code")
wdi.columns = wdi.columns.droplevel()
# NY.GDP.MKTP.KD = GDP (constant 2005 US$)
# NY.GDP.PCAP.KD = GDP per capita (constant 2005 US$)
codes = ['NY.GDP.MKTP.KD','NY.GDP.PCAP.KD', 'SP.POP.TOTL']
wdi_data = wdi[codes]
wdi_data.columns = ['egdp', 'egdppc','epop']
wdi_data = wdi_data.reset_index()
dataset = dataset.merge(wdi_data, left_on="eiso3c", right_on="iso3c")
del dataset['iso3c']
wdi_data = wdi[codes]
wdi_data.columns = ['igdp', 'igdppc','ipop']
wdi_data = wdi_data.reset_index()
dataset = dataset.merge(wdi_data, left_on="iiso3c", right_on="iso3c")
del dataset['iso3c']

In [22]:
dataset.shape

(22695, 19)

In [23]:
dataset.head()

,year,iiso3c,eiso3c,value,contig,comlang_off,colony,dist,distcap,distw,distwces,ell,ill,egdp,egdppc,epop,igdp,igdppc,ipop
0,2013,ABW,BEL,774353,0,1,0,7847.070,7847.070,7843.26,7843.01,0,0,4.204710e+11,37599.735498,11182817,NaN,NaN,102921
1,2013,ABW,BHS,4712537,0,0,0,1588.515,1588.515,1634.52,1628.14,0,0,7.835118e+09,20736.547344,377841,NaN,NaN,102921
2,2013,ABW,CHE,17812626,0,0,0,8056.332,8056.332,8074.21,8073.51,1,0,4.772463e+11,58996.896141,8089346,NaN,NaN,102921
3,2013,ABW,CHN,25319168,0,0,0,14155.350,14155.350,14590.9,14560.3,0,0,4.912954e+12,3619.439108,1357380000,NaN,NaN,102921
4,2013,ABW,COL,22160086,0,1,0,1036.634,1036.634,929.589,861.245,0,0,2.129079e+11,4497.196936,47342363,NaN,NaN,102921


In [26]:
dataset.duplicated().any()

False

In [27]:
dataset.head()

,year,iiso3c,eiso3c,value,contig,comlang_off,colony,dist,distcap,distw,distwces,ell,ill,egdp,egdppc,epop,igdp,igdppc,ipop
0,2013,ABW,BEL,774353,0,1,0,7847.070,7847.070,7843.26,7843.01,0,0,4.204710e+11,37599.735498,11182817,NaN,NaN,102921
1,2013,ABW,BHS,4712537,0,0,0,1588.515,1588.515,1634.52,1628.14,0,0,7.835118e+09,20736.547344,377841,NaN,NaN,102921
2,2013,ABW,CHE,17812626,0,0,0,8056.332,8056.332,8074.21,8073.51,1,0,4.772463e+11,58996.896141,8089346,NaN,NaN,102921
3,2013,ABW,CHN,25319168,0,0,0,14155.350,14155.350,14590.9,14560.3,0,0,4.912954e+12,3619.439108,1357380000,NaN,NaN,102921
4,2013,ABW,COL,22160086,0,1,0,1036.634,1036.634,929.589,861.245,0,0,2.129079e+11,4497.196936,47342363,NaN,NaN,102921


In [29]:
dataset[(dataset.iiso3c == "AUS")&(dataset.eiso3c=="USA")]

,year,iiso3c,eiso3c,value,contig,comlang_off,colony,dist,distcap,distw,distwces,ell,ill,egdp,egdppc,epop,igdp,igdppc,ipop
632,2013,AUS,USA,23233683745,0,1,0,16008.79,15961.95,14802,14589.2,0,0,1.445151e+13,45660.733764,316497531,8.671523e+11,37497.070617,23125868
